# Проект 3. Решить задачу DaNetQA / BoolQ

Можно решить как задачу для русского, так и для английского.

Либо провести эксперименты с многоязычной моделью

https://russiansuperglue.com/ru/tasks/task_info/DaNetQA

## Описание
Причинно-следственная связь, логический вывод, Natural Language Inference

DaNetQA - это набор да/нет вопросов с ответами и фрагментом текста, содержащим ответ. Все вопросы были написаны авторами без каких-либо искусственных ограничений.

Каждый пример представляет собой триплет (вопрос, фрагмент текста, ответ) с заголовком страницы в качестве необязательного дополнительного контекста.

Настройка классификации текстовых пар аналогична существующим задачам логического вывода (NLI)

### Тип задачи
Логика, Commonsense, Знания о мире. Бинарная классификация: true/false

# Imports

In [7]:
import pandas as pd
import unicodedata
import numpy as np

import nltk
from nltk.stem.snowball import SnowballStemmer 
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords 

In [2]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\leysh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Подготовка данных

### Загрузка данных

In [3]:
def loadJSONL(path, name):
    df = pd.read_json(path, lines=True)
    #df.drop('idx', axis=1, inplace=True)
    #df = df.set_index('idx')
    print(name)
    display(df.head())
    if (df.columns.values == 'label').any():
        s = np.unique(df['label'].to_numpy(), return_counts=True)[1]
        print(f"True answer: {s[1]}")
        print(f"False answer: {s[0]}")
        print("")
    return df

In [36]:
df_train = loadJSONL("DaNetQA/train.jsonl", "Train set")
df_validation = loadJSONL("DaNetQA/val.jsonl", "Validation set")
df_test = loadJSONL("DaNetQA/test.jsonl", "Test set:")

Train set


,question,passage,label,idx
0,Вднх - это выставочный центр?,«Вы́ставочный центр» — станция Московского мон...,True,0
1,Вднх - это выставочный центр?,"Вы́ставка достиже́ний наро́дного хозя́йства ,...",True,1
2,Был ли джиган в black star?,Вместе с этим треком они выступили на церемони...,True,2
3,Xiaomi конкурент apple?,"Xiaomi — китайская компания, основанная в 2010...",True,3
4,Был ли автомат калашникова в вов?,Отметив некоторые недостатки и в целом удачную...,False,4


True answer: 1061
False answer: 688

Validation set


,question,passage,label,idx
0,Есть ли вода на марсе?,Гидросфера Марса — это совокупность водных зап...,True,0
1,Состоит ли англия в евросоюзе?,В полночь с 31 января на 1 февраля 2020 года п...,False,1
2,Действительно ли в ссср не было адвокатов?,Семён Львович Ария — советский и российский ю...,False,2
3,Была ли чума в оране?,"Чума — это и абсурд, что осмысливается как фор...",True,3
4,Был ли кетчуп в читосе?,Текущий каталог продукции размещен на сайте пр...,True,4


True answer: 412
False answer: 409

Test set:


,question,passage,idx
0,Полезна ли ртуть с градусника?,"Отравления ртутью — расстройства здоровья, св...",0
1,Являются ли сапрофаги хищниками?,Фауна лесных почв — совокупность видов животны...,1
2,Водятся ли в индии крокодилы?,"Болотный крокодил, или магер — пресмыкающееся...",2
3,Есть ли в батате крахмал?,"Клубневидно вздутые корни весят до 15 кг, сод...",3
4,Был ли человек в железной маске?,Остров Сент-Маргерит — крупнейший из Лерински...,4


In [8]:
stemmer = SnowballStemmer('russian')

In [12]:
stemmer.stem('Собака')

'собак'

### Очистка данных

In [35]:
class DataCleaner:
    def __init__(self) -> None:
        self.stop_words = stopwords.words('russian')
        self.stemmer = SnowballStemmer('russian')

        self.count_removed_symbols = dict()
        self.count_removed_words = dict()

        self.count_replaced_symbols = dict()
        self.dict_replaced_symbols = dict()

        self.count_replaced_words = dict()
        self.dict_replaced_words = dict()

        self.char_to_remove = ['«', '»', '—', ',', '.', '-', '/', ':', '!', "?", "(", ")", "{", "}", "[", "]", "@", "#", "$", "%", "^", "&", "*", "=", "|", "\\", ">", "<"]
        self.char_to_replace = [['ё', 'е']]

    # функция подсчета количества измененных слов
    def addReplacedWord(self, s_from, s_to = ' '):
        if not self.count_replaced_words.keys().__contains__(s_from):
            self.count_replaced_words[s_from] = 0
        self.count_replaced_words[s_from] += 1
        self.dict_replaced_words[s_from] = s_to

    # функция подсчета количества удаленных слов
    def addRemovedWord(self, w):
        if w == ' ':
            if not self.count_removed_symbols.keys().__contains__(w):
                self.count_removed_symbols[w] = 0
            self.count_removed_symbols[w] += 1

    # функция подсчета количества удаленных символов
    def addReplacedSymbol(self, s_from, s_to = ' '):
        if s_to == ' ':
            if not self.count_removed_symbols.keys().__contains__(s_from):
                self.count_removed_symbols[s_from] = 0
            self.count_removed_symbols[s_from] += 1
        else:
            if not self.count_replaced_symbols.keys().__contains__(s_from):
                self.count_replaced_symbols[s_from] = 0
            self.count_replaced_symbols[s_from] += 1
            self.dict_replaced_symbols[s_from] = s_to

    # удаление знаков ударения и прочих символов unicode
    def unicodeToAscii(self, s):
        tmp = []
        for c in unicodedata.normalize('NFD', s):
            if unicodedata.category(c) != 'Mn':
                tmp.append(c)
            else:
                self.addReplacedSymbol(c)
        return ''.join(tmp)

    # если нужно удалить, то заменяем на пробел чтоб не потерят разделения слов
    def replaceChar(self, s):
        tmp = []
        for i, c in enumerate(s):
            if self.char_to_remove.__contains__(c):
                self.addReplacedSymbol(c, s[i])
                tmp.append(' ')
            else:
                tmp.append(c)
        s = "".join(tmp)

        for s_from, s_to in self.char_to_replace:
            if c == s_from:
                s[i] = s_to
                self.addReplacedSymbol(s_from, s_to)
        return s

    # удаляем лишние пробелы
    def trimSpaces(self, s):
        while s.__contains__('  '):
            s = s.replace('  ', ' ')
        s = s.strip()
        return s

    # удаляем слва из stopwords
    def removeStopWords(self, s):
        tmp = []
        for word in word_tokenize(s):
            if word not in self.stop_words:
                tmp.append(word)
            else:
                self.addRemovedWord(word)
        return " ".join(tmp)

    # удаляем слва из stopwords
    def StemmWords(self, s):
        tmp = []
        for word in word_tokenize(s):
            wordStemmed = self.stemmer.stem(word)
            tmp.append(wordStemmed)
            if word != wordStemmed:
                self.addReplacedWord(word, wordStemmed)
        return " ".join(tmp)

    def clean(self, df, column):
        for i in range(len(df)):
            df[column][i] = self.unicodeToAscii(df[column][i])
            df[column][i] = df[column][i].lower()
            df[column][i] = self.replaceChar(df[column][i])
            df[column][i] = self.removeStopWords(df[column][i])
            df[column][i] = self.StemmWords(df[column][i])
            df[column][i] = self.trimSpaces(df[column][i])
        return df

    def summary(self):
        dfs = []

        print("===================================")
        print("========= Data Cleaner log ========")
        print("===================================")
        print("===                             ===")
        print("===        Removed Chars        ===")
        print("===                             ===")
        print("===================================")
        
        cols = ["symbol", "count_removed"]
        dfRemoved = pd.DataFrame(columns=cols)
        for c in self.count_removed_symbols:
            current_df = pd.DataFrame([[c, self.count_removed_symbols[c]]], columns=cols) 
            dfRemoved = pd.concat([dfRemoved, current_df], ignore_index=True)
        display(dfRemoved)
        dfs.append(dfRemoved)

        print("===================================")
        print("===                             ===")
        print("===        Removed Words        ===")
        print("===                             ===")
        print("===================================")
        
        cols = ["word", "count_removed"]
        dfRemoved = pd.DataFrame(columns=cols)
        for c in self.count_removed_words:
            current_df = pd.DataFrame([[c, self.count_removed_words[c]]], columns=cols) 
            dfRemoved = pd.concat([dfRemoved, current_df], ignore_index=True)
        display(dfRemoved)
        dfs.append(dfRemoved)

        print("===================================")
        print("===                             ===")
        print("===        Replaced Chars       ===")
        print("===                             ===")
        print("===================================")
        
        cols = ["symbol_from", "symbol_to", "count_replaced"]
        dfRemoved = pd.DataFrame(columns=cols)
        for c in self.dict_replaced_symbols:
            current_df = pd.DataFrame([[ c, self.dict_replaced_symbols[c], self.count_replaced_symbols[c]]], columns=cols) 
            dfRemoved = pd.concat([dfRemoved, current_df], ignore_index=True)
        display(dfRemoved)
        dfs.append(dfRemoved)

        print("===================================")
        print("===                             ===")
        print("===        Stemmed Words        ===")
        print("===                             ===")
        print("===================================")
        
        cols = ["word_from", "word_to", "count_replaced"]
        dfRemoved = pd.DataFrame(columns=cols)
        for c in self.dict_replaced_words:
            current_df = pd.DataFrame([[ c, self.dict_replaced_words[c], self.count_replaced_words[c]]], columns=cols) 
            dfRemoved = pd.concat([dfRemoved, current_df], ignore_index=True)
        display(dfRemoved)
        dfs.append(dfRemoved)

        return dfs


In [37]:
t = DataCleaner()
df_train = t.clean(df_train, 'passage')
df_test = t.clean(df_test, 'passage')
df_validation = t.clean(df_validation, 'passage')
df_train = t.clean(df_train, 'question')
df_test = t.clean(df_test, 'question')
df_validation = t.clean(df_validation, 'question')
dfs = t.summary()

C:\Users\leysh\AppData\Local\Temp\ipykernel_29156\3942620269.py:100: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column][i] = self.unicodeToAscii(df[column][i])
C:\Users\leysh\AppData\Local\Temp\ipykernel_29156\3942620269.py:101: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column][i] = df[column][i].lower()
C:\Users\leysh\AppData\Local\Temp\ipykernel_29156\3942620269.py:102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[co

========= Data Cleaner log ========
===                             ===
===        Removed Chars        ===
===                             ===


,symbol,count_removed
0,́,1618
1,̆,27745
2,̈,5295
3,̔,5
4,̀,6
5,̣,6
6,̌,1
7,̥,1
8,̂,4
9,̊,2


===                             ===
===        Removed Words        ===
===                             ===


,word,count_removed


===                             ===
===        Replaced Chars       ===
===                             ===


,symbol_from,symbol_to,count_replaced
0,«,«,2246
1,»,»,2233
2,—,—,4524
3,.,.,19367
4,-,-,4010
5,",",",",27875
6,),),1430
7,:,:,1395
8,%,%,348
9,/,/,198


===                             ===
===        Stemmed Words        ===
===                             ===


,word_from,word_to,count_replaced
0,выставочныи,выставочны,7
1,станция,станц,7
2,московского,московск,26
3,монорельса,монорельс,2
4,расположена,располож,21
...,...,...,...
50960,мусульманину,мусульманин,1
50961,христианке,христианк,1
50962,колоннады,колоннад,1
50963,ростральных,ростральн,1


In [19]:
df_train.to_json("DaNetQA/train_c.jsonl", force_ascii=False, lines=True, orient='records')
df_test.to_json("DaNetQA/test_c.jsonl", force_ascii=False, lines=True, orient='records')
df_validation.to_json("DaNetQA/val_c.jsonl", force_ascii=False, lines=True, orient='records')

In [45]:
df=dfs[3]

In [50]:
df

,word_from,word_to,count_replaced
0,выставочныи,выставочны,7
1,станция,станц,7
2,московского,московск,26
3,монорельса,монорельс,2
4,расположена,располож,21
...,...,...,...
50960,мусульманину,мусульманин,1
50961,христианке,христианк,1
50962,колоннады,колоннад,1
50963,ростральных,ростральн,1


In [52]:
df.sort_values(by=['count_replaced'],ascending=False)

,word_from,word_to,count_replaced
32,года,год,1802
102,году,год,1101
140,также,такж,984
159,является,явля,709
152,время,врем,689
...,...,...,...
32624,триплоидная,триплоидн,1
32623,аутосом,аутос,1
16557,туман,тума,1
16559,подробностеи,подробн,1


# Model

In [20]:
import codecs
import json
from sklearn.linear_model import LogisticRegression
import pickle
import joblib

## LogisticRegression Baseline

In [21]:
def save_output(data, path):
    with open(path, mode="w") as file:
        for line in sorted(data, key=lambda x: int(x.get("idx"))):
            line["idx"] = int(line["idx"])
            file.write(f"{json.dumps(line, ensure_ascii=False)}\n")

In [22]:
def build_feature_DaNetQA(row):
    res = str(row["question"]).strip()
    label = row.get("label")
    return res, label

In [23]:
def build_features_DaNetQA(path, vect):
    with codecs.open(path, encoding='utf-8-sig') as reader:
        lines = reader.read().split("\n")
        lines = list(map(json.loads, filter(None, lines)))
    res = list(map(build_feature_DaNetQA, lines))
    texts = list(map(lambda x: x[0], res))
    labels = list(map(lambda x: x[1], res))
    ids = [x["idx"] for x in lines]
    return (vect.transform(texts), labels), ids

In [24]:
def fit_DaNetQA(train, labels):
    clf = LogisticRegression()
    return clf.fit(train, labels)

In [25]:
def eval_DaNetQA(train_path, val_path, test_path, vect):
    train, _ = build_features_DaNetQA(train_path, vect)
    val, _ = build_features_DaNetQA(val_path, vect)
    test, ids = build_features_DaNetQA(test_path, vect)
    clf = fit_DaNetQA(*train)
    try:
        test_score = clf.score(*test)
    except ValueError:
        test_score = None
    test_pred = clf.predict(test[0])
    return clf, {
        "train": clf.score(*train),
        "val": clf.score(*val),
        "test": test_score,
        "test_pred": [{"idx": idx, "label": str(label).lower()} for idx, label in zip(ids, test_pred)]
    }

## Load Model

In [ ]:
!wget https://russiansuperglue.com/tasks/tf_idf
!unzip tf_idf_baseline.zip
!rm tf_idf_baseline.zip

In [26]:
vect = joblib.load("tfidf.pkl")

c:\Users\leysh\anaconda3\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.21.3 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\leysh\anaconda3\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.21.3 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


## Score Model

In [27]:
train_path = "DaNetQA/train_c.jsonl"
val_path = "DaNetQA/val_c.jsonl"
test_path = "DaNetQA/test_c.jsonl"

In [28]:
_, DaNetQA_scores = eval_DaNetQA(train_path, val_path, test_path, vect)

In [31]:
DaNetQA_scores["train"], DaNetQA_scores["val"]

(0.7004002287021155, 0.5371498172959805)

In [33]:
DaNetQA_scores

{'train': 0.7004002287021155,
 'val': 0.5371498172959805,
 'test': None,
 'test_pred': [{'idx': 0, 'label': 'true'},
  {'idx': 1, 'label': 'true'},
  {'idx': 2, 'label': 'true'},
  {'idx': 3, 'label': 'true'},
  {'idx': 4, 'label': 'true'},
  {'idx': 5, 'label': 'true'},
  {'idx': 6, 'label': 'true'},
  {'idx': 7, 'label': 'true'},
  {'idx': 8, 'label': 'true'},
  {'idx': 9, 'label': 'true'},
  {'idx': 10, 'label': 'true'},
  {'idx': 11, 'label': 'true'},
  {'idx': 12, 'label': 'true'},
  {'idx': 13, 'label': 'true'},
  {'idx': 14, 'label': 'true'},
  {'idx': 15, 'label': 'true'},
  {'idx': 16, 'label': 'false'},
  {'idx': 17, 'label': 'true'},
  {'idx': 18, 'label': 'true'},
  {'idx': 19, 'label': 'false'},
  {'idx': 20, 'label': 'true'},
  {'idx': 21, 'label': 'true'},
  {'idx': 22, 'label': 'true'},
  {'idx': 23, 'label': 'true'},
  {'idx': 24, 'label': 'true'},
  {'idx': 25, 'label': 'false'},
  {'idx': 26, 'label': 'true'},
  {'idx': 27, 'label': 'true'},
  {'idx': 28, 'label': 't

# Fine tune

In [54]:
from datasets import load_dataset

dataset = load_dataset("yelp_review_full")
dataset["train"][100]

Generating train split:   0%|          | 0/650000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset yelp_review_full downloaded and prepared to C:/Users/leysh/.cache/huggingface/datasets/yelp_review_full/yelp_review_full/1.0.0/e8e18e19d7be9e75642fc66b198abadb116f73599ec89a69ba5dd8d1e57ba0bf. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

{'label': 0,
 'text': 'My expectations for McDonalds are t rarely high. But for one to still fail so spectacularly...that takes something special!\\nThe cashier took my friends\'s order, then promptly ignored me. I had to force myself in front of a cashier who opened his register to wait on the person BEHIND me. I waited over five minutes for a gigantic order that included precisely one kid\'s meal. After watching two people who ordered after me be handed their food, I asked where mine was. The manager started yelling at the cashiers for \\"serving off their orders\\" when they didn\'t have their food. But neither cashier was anywhere near those controls, and the manager was the one serving food to customers and clearing the boards.\\nThe manager was rude when giving me my order. She didn\'t make sure that I had everything ON MY RECEIPT, and never even had the decency to apologize that I felt I was getting poor service.\\nI\'ve eaten at various McDonalds restaurants for over 30 years. 

In [55]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

c:\Users\leysh\anaconda3\lib\site-packages\huggingface_hub\file_download.py:123: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\leysh\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436k [00:00<?, ?B/s]

  0%|          | 0/650 [00:00<?, ?ba/s]

  0%|          | 0/50 [00:00<?, ?ba/s]

In [57]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

In [56]:
#@title
from IPython.display import HTML

HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/nvBXf7s7vTI?rel=0&amp;controls=0&amp;showinfo=0" frameborder="0" allowfullscreen></iframe>')


c:\Users\leysh\anaconda3\lib\site-packages\IPython\core\display.py:724: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


In [58]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)

Downloading:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [59]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")

In [60]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

C:\Users\leysh\AppData\Local\Temp\ipykernel_29156\2607597888.py:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")


In [61]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [62]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

In [63]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

In [64]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
c:\Users\leysh\anaconda3\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1000
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 375


  0%|          | 0/375 [00:00<?, ?it/s]

The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 8


  0%|          | 0/125 [00:00<?, ?it/s]

{'eval_loss': 1.1737103462219238, 'eval_accuracy': 0.477, 'eval_runtime': 14.5774, 'eval_samples_per_second': 68.599, 'eval_steps_per_second': 8.575, 'epoch': 1.0}


...

The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 8


  0%|          | 0/125 [00:00<?, ?it/s]



Training completed. Do not forget to share your model on huggingface.co/models =)




{'eval_loss': 1.0267807245254517, 'eval_accuracy': 0.556, 'eval_runtime': 15.4155, 'eval_samples_per_second': 64.87, 'eval_steps_per_second': 8.109, 'epoch': 3.0}
{'train_runtime': 182.9724, 'train_samples_per_second': 16.396, 'train_steps_per_second': 2.049, 'train_loss': 1.0788536783854166, 'epoch': 3.0}


TrainOutput(global_step=375, training_loss=1.0788536783854166, metrics={'train_runtime': 182.9724, 'train_samples_per_second': 16.396, 'train_steps_per_second': 2.049, 'train_loss': 1.0788536783854166, 'epoch': 3.0})

In [65]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator(return_tensors="tf")

In [66]:
tf_train_dataset = small_train_dataset.to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["labels"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)

tf_validation_dataset = small_eval_dataset.to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["labels"],
    shuffle=False,
    collate_fn=data_collator,
    batch_size=8,
)

ImportError: Called a Tensorflow-specific function but Tensorflow is not installed.